In [ ]:
def _something(o: Any) -> bytes:
    if hasattr(o, "json"):
        return o.json().encode("utf-8")
    else:
        return json.dumps(o).encode("utf-8")
        
_something(MyInfo(mobile="123", name="ja")), _something({"a": 123})

In [ ]:
producer = AIOProducer()

@app.produces(topic="topic_5", producer=producer)
async def f5(addr: str, port: int) -> ProduceMessage[MyMsgUrl]:
    response = ProduceMessage(msg=MyMsgUrl(url=f"https://{addr}:{port}"), key=addr)
    return response

@app.produces(topic="topic_6", producer=producer)
async def f6(addr: str, port: int) -> ProduceMessage[MyMsgUrl]:
    response = ProduceMessage(msg=MyMsgUrl(url=f"https://{addr}:{port}"), key=port)
    return response

@app.produces_on_error(topic="error_topic", producer=producer)
async def ferr(err: Exception) -> ProduceMessage[MyMsgUrl]:
    response = ProduceMessage(err)
    return response

@app.consumes(topic="topic_7"):
async def g(msg: MyMsgUrl, produce: ProduceCallable):
    addr = msg.url
    port = 8000
    await produce(topic="topic_6", addr=addr, port=port)
    
@app.consumes(topic="topic_7"):
async def g(msg: MyMsgUrl):
    addr = msg.url
    port = 8000
    await f6(addr, port)

In [ ]:
from typing import TypeVar, Generic
from logging import Logger

T = TypeVar('T')

class KafkaProduceMessage(Generic[T]):
    def __init__(self, value: T, key: Optional[Any] = None, raw_key: Optional[bytes] = None) -> None:
        self.value = value
        if key is None:
            self.raw_key = raw_key
        else:
            if raw_key is not None:
                raise ValueError("At most one of key and raw_key can be non-None, we have key='{key}' and raw_key='{raw_key}'")
                
            if hasattr(key, "json"):
                self.raw_key = key.json().encode("utf-8")
            else:
                self.raw_key = json.dumps(key).encode("utf-8")

    def set(self, new: T) -> None:
        self.value = new

    def get(self) -> T:
        return self.value
    
    def __repr__(self):
        kwargs = ", ".join([f'{k}={v}' for k, v in self.__dict__.items() if not k.startswith("_")])
        return f"{self.__class__.__name__}({kwargs})"

In [ ]:
KafkaProduceMessage[str]("James Bond")

In [ ]:
signature_item = KafkaProduceMessage[MyMsgEmail]
t = signature_item.__args__ [0]
t

In [ ]:
# don't wait for specs to be generated (takes 10 sec or so)

T = TypeVar('T')

with unittest.mock.patch("__main__.export_async_spec"):
    
    # mock up send method of AIOKafkaProducer
    with mock_AIOKafkaProducer_send() as mock:

        app = create_testing_app()
        
        @app.consumes()
        def on_my_input_topic(msg: MyMsgUrl) -> None:
            msg_email = MyMsgEmail(msg_url=msg_url, msg_email="someone@acme.com")
            to_my_output_topic(msg_email)
        
        @app.consumes(max_poll_records=32, )
        def on_my_input_topic(msgs: List[MyMsgUrl]) -> None:
            out_msgs = [MyMsgEmail(msg_url=msg_url, msg_email="someone@acme.com") for msg_ulr in msgs]
            to_my_output_topic(out_msgs)
        
        @app.produces()
        def to_my_output_topic(url: List[str]) -> List[Tuple[MyMsgUrl], str]:
            pass
        
        @app.consumes(topic="input_topic", bootstrap_servers="in_kafka.acme.com")
        @app.produces(topic="output_topic", bootstrap_servers="out_kafka.acme.com")
        def pipe_1(msg_url: MyMsgUrl) -> MyMsgEmail:
            msg_email = MyMsgEmail(msg_url=msg_url, msg_email="someone@acme.com")
            return msg_email
        
        @app.transforms(
            in_topic="input_topic",
            out_topic="output_topic",
        )
        def pipe_2(msg_url: MyMsgUrl) -> Tuple[MyMsgEmail, str]:
            msg_email = MyMsgEmail(msg_url=msg_url, msg_email="someone@acme.com")
            return msg_email, msg_url.url
        
        @app.produces()
        def to_my_test_topic(mobile: str, url: str) -> MyMsgUrl:
            msg = MyMsgUrl(
                info=dict(mobile=mobile, name="James Bond"), url=url
            )
            return msg

        @app.produces(use_key=True)
        def to_my_test_topic_with_key(mobile: str, url: str) -> Tuple[MyMsgUrl, Any]:
            msg = MyMsgUrl(
                info=dict(mobile=mobile, name="James Bond"), url=url
            )
            return msg, url

        try:
            app._on_startup()
            await to_my_test_topic(mobile="+385912345678", url="https://www.vip.hr")
            await to_my_test_topic_with_key(mobile="+385987654321", url="https://www.ht.hr")
        finally:
            await app._on_shutdown()

        mock.assert_called_with(
            "my_test_topic",
            b'{"info": {"mobile": "+385912345678", "name": "James Bond"}, "url": "https://www.vip.hr"}',
        )
        mock.my_test_topic_with_key(
            "my_test_topic",
            b'{"info": {"mobile": "+385987654321", "name": "James Bond"}, "url": "https://www.ht.hr"}',
            key=b"",
        )
        

In [ ]:
# TODO: Test produce on consume function
app = create_testing_app()

@app.consumes(auto_offset_reset="latest")
@app.produces(topic="redirect_topic")
def on_my_test_topic(msg: MyMsgUrl) -> MyMsgUrl:
    return msg

try:
    app._on_startup()
    await asyncio.sleep(60)
finally:
    await app._on_shutdown()

In [ ]:
# | export


@patch
def produce_raw(
    self: FastKafkaAPI,
    topic: str,
    raw_msg: Union[str, bytes],
    on_delivery: Optional[Callable[[KafkaMessage, Message], None]] = None,
) -> "asyncio.Future[Any]":

    if isinstance(raw_msg, str):
        raw_msg = raw_msg.encode("utf-8")

    if on_delivery is None:
        on_delivery = self._store["producers"][topic]  # type: ignore

    if iscoroutinefunction(on_delivery):
        raise ValueError("coroutines not supported for callbacks yet")

    p: AIOProducer = self._confluent_producer  # type: ignore

    def _delivery_report(
        kafka_err: KafkaError,
        kafka_msg: Message,
        self=self,
        topic=topic,
        raw_msg=raw_msg,
        on_delivery=on_delivery,
    ):
        msg_cls: KafkaMessage
        if kafka_err is not None:
            logger.info(f"produce_raw() topic={topic} raw_msg={raw_msg} delivery error")
            if self._on_error_topic is not None:
                on_error = self._store["producers"][self._on_error_topic]
                msg_cls = _get_msg_cls_for_method(on_error)
                on_error(
                    msg_cls("Message delivery failed: {}".format(kafka_err)), kafka_err  # type: ignore
                )
        else:
            logger.info(f"produce_raw() topic={topic} raw_msg={raw_msg} delivered")
            msg_cls = _get_msg_cls_for_method(on_delivery)
            on_delivery(msg_cls.parse_raw(raw_msg), kafka_msg)

    return p.send(topic, raw_msg)

In [ ]:
raw_msg = (
    MyMsgUrl(
        info=dict(mobile="+385987654321", name="James Bond"),
        url="https://sis.gov.uk/agents/007",
    )
    .json()
    .encode("utf-8")
)


async def test_me():
    async with start_test_app() as app:
        await app.produce_raw("my_topic_3", raw_msg)

        def _on_delivery(msg: KafkaMessage, *args):
            logger.warning("me so cool")

        # we don't need to wait for it
        app.produce_raw("my_topic_3", raw_msg, on_delivery=_on_delivery)


asyncio.run(test_me())


print("ok")

In [ ]:
# | export


@patch
def test_run(self: FastKafkaAPI, f: Callable[[], Any], timeout: int = 30):
    async def _loop(app: FastKafkaAPI = self, f: Callable[[], Any] = f):
        logger.info(f"test_run(): starting")
        try:
            async with anyio.create_task_group() as tg:
                with anyio.move_on_after(timeout) as scope:
                    app._on_startup()  # type: ignore

                    if iscoroutinefunction(f):
                        logger.info(f"test_run(app={app}, f={f}): Calling coroutine {f}")
                        retval = await f()
                    else:
                        logger.info(f"test_run(app={app}, f={f}): Calling function {f}")
                        retval = await asyncer.asyncify(f)()

                return retval
        except Exception as e:
            logger.error(f"test_run(): exception caugth {e}")
            raise e
        finally:
            logger.info(f"test_run(app={app}, f={f}): shutting down the app")
            await app._on_shutdown()
            logger.info(f"test_run(app={app}, f={f}): finished")

    return asyncer.runnify(_loop)()

In [ ]:
# |export


@patch
@asynccontextmanager
async def testing_ctx(self: FastKafkaAPI, timeout: int = 30):
    logger.info(f"test_context(): starting")
    try:
        async with anyio.create_task_group() as tg:
            with anyio.move_on_after(timeout) as scope:
                self._on_startup()  # type: ignore

                yield

    except Exception as e:
        logger.error(f"test_context(): exception caugth {e}")
        raise e
    finally:
        logger.info(f"test_context(self={self}): shutting down the app")
        await self._on_shutdown()
        logger.info(f"test_context(self={self}): finished")

In [ ]:
async with app.testing_ctx():
    print("app is up and running")
    await anyio.sleep(2)
print("app is shut down")

In [ ]:
async with app.testing_ctx(timeout=3):
    print("app is up and running")
    await anyio.sleep(1000)
print("app is shut down")

In [ ]:
# | eval: false

# uvicorn.run(app, host="0.0.0.0", port=6006)